<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/EVM_Solana_Bridge_onSolana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
use borsh::{BorshDeserialize, BorshSerialize};
use solana_program::{
    account_info::{next_account_info, AccountInfo},
    entrypoint,
    entrypoint::ProgramResult,
    msg,
    program::invoke_signed,
    program_error::ProgramError,
    pubkey::Pubkey,
    rent::Rent,
    system_instruction,
    sysvar::Sysvar,
};
use spl_token::{
    instruction as token_instruction,
    state::{Account as TokenAccount, Mint},
};

// Define the program ID
solana_program::declare_id!("Your_Program_ID_Here");

#[derive(BorshSerialize, BorshDeserialize, Debug)]
pub struct BridgeConfig {
    pub authority: Pubkey,
    pub oracle: Pubkey,
    pub nonce: u64,
}

#[derive(BorshSerialize, BorshDeserialize, Debug)]
pub struct BridgeRequest {
    pub eth_sender: [u8; 20],
    pub solana_recipient: Pubkey,
    pub token_mint: Pubkey,
    pub amount: u64,
    pub nonce: u64,
    pub processed: bool,
}

#[derive(BorshSerialize, BorshDeserialize, Debug)]
pub enum BridgeInstruction {
    // Initialize the bridge
    Initialize {
        authority: Pubkey,
        oracle: Pubkey,
    },
    // Process a bridge request from Ethereum
    ProcessBridgeRequest {
        eth_sender: [u8; 20],
        amount: u64,
        eth_nonce: u64,
    },
    // Set a new oracle
    SetOracle {
        new_oracle: Pubkey,
    },
    // Update authority
    UpdateAuthority {
        new_authority: Pubkey,
    },
}

// Program entrypoint
entrypoint!(process_instruction);

// Program logic
fn process_instruction(
    program_id: &Pubkey,
    accounts: &[AccountInfo],
    instruction_data: &[u8],
) -> ProgramResult {
    let instruction = BridgeInstruction::try_from_slice(instruction_data)
        .map_err(|_| ProgramError::InvalidInstructionData)?;

    match instruction {
        BridgeInstruction::Initialize { authority, oracle } => {
            process_initialize(program_id, accounts, authority, oracle)
        }
        BridgeInstruction::ProcessBridgeRequest {
            eth_sender,
            amount,
            eth_nonce,
        } => process_bridge_request(program_id, accounts, eth_sender, amount, eth_nonce),
        BridgeInstruction::SetOracle { new_oracle } => {
            process_set_oracle(program_id, accounts, new_oracle)
        }
        BridgeInstruction::UpdateAuthority { new_authority } => {
            process_update_authority(program_id, accounts, new_authority)
        }
    }
}

fn process_initialize(
    program_id: &Pubkey,
    accounts: &[AccountInfo],
    authority: Pubkey,
    oracle: Pubkey,
) -> ProgramResult {
    let account_info_iter = &mut accounts.iter();

    // Get accounts
    let initializer = next_account_info(account_info_iter)?;
    let bridge_config_account = next_account_info(account_info_iter)?;
    let system_program = next_account_info(account_info_iter)?;

    // Verify signer
    if !initializer.is_signer {
        return Err(ProgramError::MissingRequiredSignature);
    }

    // Create bridge config account
    let rent = Rent::get()?;
    let space = BridgeConfig::default().try_to_vec()?.len();
    let lamports = rent.minimum_balance(space);

    invoke_signed(
        &system_instruction::create_account(
            initializer.key,
            bridge_config_account.key,
            lamports,
            space as u64,
            program_id,
        ),
        &[
            initializer.clone(),
            bridge_config_account.clone(),
            system_program.clone(),
        ],
        &[],
    )?;

    // Initialize bridge config
    let bridge_config = BridgeConfig {
        authority,
        oracle,
        nonce: 0,
    };

    bridge_config.serialize(&mut *bridge_config_account.data.borrow_mut())?;

    msg!("Bridge initialized");
    Ok(())
}

fn process_bridge_request(
    program_id: &Pubkey,
    accounts: &[AccountInfo],
    eth_sender: [u8; 20],
    amount: u64,
    eth_nonce: u64,
) -> ProgramResult {
    let account_info_iter = &mut accounts.iter();

    // Get accounts
    let oracle = next_account_info(account_info_iter)?;
    let bridge_config_account = next_account_info(account_info_iter)?;
    let request_account = next_account_info(account_info_iter)?;
    let recipient = next_account_info(account_info_iter)?;
    let recipient_token_account = next_account_info(account_info_iter)?;
    let mint = next_account_info(account_info_iter)?;
    let bridge_authority = next_account_info(account_info_iter)?;
    let token_program = next_account_info(account_info_iter)?;
    let system_program = next_account_info(account_info_iter)?;

    // Verify signer
    if !oracle.is_signer {
        return Err(ProgramError::MissingRequiredSignature);
    }

    // Load bridge config
    let mut bridge_config = BridgeConfig::try_from_slice(&bridge_config_account.data.borrow())?;

    // Verify oracle
    if *oracle.key != bridge_config.oracle {
        return Err(ProgramError::InvalidAccountData);
    }

    // Create request account if it doesn't exist
    if request_account.data_is_empty() {
        let rent = Rent::get()?;
        let space = BridgeRequest::default().try_to_vec()?.len();
        let lamports = rent.minimum_balance(space);

        invoke_signed(
            &system_instruction::create_account(
                oracle.key,
                request_account.key,
                lamports,
                space as u64,
                program_id,
            ),
            &[
                oracle.clone(),
                request_account.clone(),
                system_program.clone(),
            ],
            &[],
        )?;
    }

    // Initialize bridge request
    let bridge_request = BridgeRequest {
        eth_sender,
        solana_recipient: *recipient.key,
        token_mint: *mint.key,
        amount,
        nonce: eth_nonce,
        processed: false,
    };

    bridge_request.serialize(&mut *request_account.data.borrow_mut())?;

    // Derive PDA for bridge authority
    let (pda, bump_seed) = Pubkey::find_program_address(&[b"bridge_authority"], program_id);
    if pda != *bridge_authority.key {
        return Err(ProgramError::InvalidAccountData);
    }

    // Mint tokens to recipient
    invoke_signed(
        &token_instruction::mint_to(
            token_program.key,
            mint.key,
            recipient_token_account.key,
            bridge_authority.key,
            &[],
            amount,
        )?,
        &[
            mint.clone(),
            recipient_token_account.clone(),
            bridge_authority.clone(),
            token_program.clone(),
        ],
        &[&[b"bridge_authority", &[bump_seed]]],
    )?;

    // Update bridge nonce
    bridge_config.nonce = bridge_config.nonce.saturating_add(1);
    bridge_config.serialize(&mut *bridge_config_account.data.borrow_mut())?;

    // Mark request as processed
    let mut request = BridgeRequest::try_from_slice(&request_account.data.borrow())?;
    request.processed = true;
    request.serialize(&mut *request_account.data.borrow_mut())?;

    msg!("Bridge request processed: Ethereum sender: {:?}, Solana recipient: {}, amount: {}",
        eth_sender, recipient.key, amount);
    Ok(())
}

fn process_set_oracle(
    _program_id: &Pubkey,
    accounts: &[AccountInfo],
    new_oracle: Pubkey,
) -> ProgramResult {
    let account_info_iter = &mut accounts.iter();

    // Get accounts
    let authority = next_account_info(account_info_iter)?;
    let bridge_config_account = next_account_info(account_info_iter)?;

    // Verify signer
    if !authority.is_signer {
        return Err(ProgramError::MissingRequiredSignature);
    }

    // Load bridge config
    let mut bridge_config = BridgeConfig::try_from_slice(&bridge_config_account.data.borrow())?;

    // Verify authority
    if *authority.key != bridge_config.authority {
        return Err(ProgramError::InvalidAccountData);
    }

    // Update oracle
    bridge_config.oracle = new_oracle;
    bridge_config.serialize(&mut *bridge_config_account.data.borrow_mut())?;

    msg!("Oracle updated to {}", new_oracle);
    Ok(())
}

fn process_update_authority(
    _program_id: &Pubkey,
    accounts: &[AccountInfo],
    new_authority: Pubkey,
) -> ProgramResult {
    let account_info_iter = &mut accounts.iter();

    // Get accounts
    let authority = next_account_info(account_info_iter)?;
    let bridge_config_account = next_account_info(account_info_iter)?;

    // Verify signer
    if !authority.is_signer {
        return Err(ProgramError::MissingRequiredSignature);
    }

    // Load bridge config
    let mut bridge_config = BridgeConfig::try_from_slice(&bridge_config_account.data.borrow())?;

    // Verify authority
    if *authority.key != bridge_config.authority {
        return Err(ProgramError::InvalidAccountData);
    }

    // Update authority
    bridge_config.authority = new_authority;
    bridge_config.serialize(&mut *bridge_config_account.data.borrow_mut())?;

    msg!("Authority updated to {}", new_authority);
    Ok(())
}

// Default implementations for Borsh serialization
impl Default for BridgeConfig {
    fn default() -> Self {
        Self {
            authority: Pubkey::default(),
            oracle: Pubkey::default(),
            nonce: 0,
        }
    }
}

impl Default for BridgeRequest {
    fn default() -> Self {
        Self {
            eth_sender: [0; 20],
            solana_recipient: Pubkey::default(),
            token_mint: Pubkey::default(),
            amount: 0,
            nonce: 0,
            processed: false,
        }
    }
}